In [1]:
#! pip install -U climetlab --quiet
#! pip install -U climetlab_s2s_ai_challenge --quiet

In [2]:
import climetlab as cml 

/home/mafp/.conda/envs/dev8/lib/python3.8/site-packages/ecmwflibs.libs/libMagPlus-887d8f5c.so


In [3]:
import climetlab_s2s_ai_challenge
print(f'Climetlab version : {cml.__version__}')
print(f'Climetlab-s2s-ai-challenge plugin version : {climetlab_s2s_ai_challenge.__version__}')

Climetlab version : 0.5.6
Climetlab-s2s-ai-challenge plugin version : 0.4.16


# Observations data for training

Climetlab provides the observation datasets. They can be used as a xarray.Dataset :

In [4]:
cmlds = cml.load_dataset("s2s-ai-challenge-training-output-reference",
                         date = 20200102,
                         parameter='t2m'                         
                       )
cmlds.to_xarray()

By downloading data from this dataset, you agree to the terms and conditions defined at https://apps.ecmwf.int/datasets/data/s2s/licence/. If you do not agree with such terms, do not download the data.  This dataset has been dowloaded from IRIDL. By downloading this data you also agree to the terms and conditions defined at https://iridl.ldeo.columbia.edu.


<xarray.Dataset>
Dimensions:        (forecast_time: 20, latitude: 19, lead_time: 46, longitude: 36)
Coordinates:
    valid_time     (lead_time, forecast_time) datetime64[ns] ...
  * latitude       (latitude) float64 90.0 80.0 70.0 60.0 ... -70.0 -80.0 -90.0
  * longitude      (longitude) float64 0.0 10.0 20.0 30.0 ... 330.0 340.0 350.0
  * forecast_time  (forecast_time) datetime64[ns] 2000-01-02 ... 2019-01-02
  * lead_time      (lead_time) timedelta64[ns] 0 days 1 days ... 44 days 45 days
Data variables:
    t2m            (lead_time, forecast_time, latitude, longitude) float64 ...
Attributes:
    source_dataset_name:  temperature daily from NOAA NCEP CPC: Climate Predi...
    source_hosting:       IRIDL
    source_url:           http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCEP/...
    created_by_person:    Florian Pinault Florian.Pinault@ecmwf.int and Aaron...
    created_by_software:  climetlab-s2s-ai-challenge
    created_by_script:    tools/build_dataset_observations.py
    timestamp:            11/05/2021 14:34:28

In [5]:
cmlds = cml.load_dataset("s2s-ai-challenge-training-output-reference",
                         date = 20200102,
                         parameter='tp'                         
                       )
cmlds.to_xarray()

<xarray.Dataset>
Dimensions:        (forecast_time: 20, latitude: 19, lead_time: 46, longitude: 36)
Coordinates:
  * latitude       (latitude) float64 90.0 80.0 70.0 60.0 ... -70.0 -80.0 -90.0
  * longitude      (longitude) float64 0.0 10.0 20.0 30.0 ... 330.0 340.0 350.0
  * forecast_time  (forecast_time) datetime64[ns] 2000-01-02 ... 2019-01-02
  * lead_time      (lead_time) timedelta64[ns] 0 days 1 days ... 44 days 45 days
    valid_time     (lead_time, forecast_time) datetime64[ns] ...
Data variables:
    tp             (lead_time, forecast_time, latitude, longitude) float64 ...
Attributes:
    source_dataset_name:  NOAA NCEP CPC UNIFIED_PRCP GAUGE_BASED GLOBAL v1p0 ...
    source_hosting:       IRIDL
    source_url:           http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCEP/...
    created_by_person:    Florian Pinault Florian.Pinault@ecmwf.int and Aaron...
    created_by_software:  climetlab-s2s-ai-challenge
    created_by_script:    tools/build_dataset_observations.py
    timestamp:            11/05/2021 14:34:55